In [4]:
import os
import random
import numpy as np
import pandas as pd
import math

from glob import glob
from IPython.display import display

#### Read dataset

In [5]:
#Defining original dataset directory.
Path = 'c://Datasets//Multiagent//rtb'

#Importing data.
os.chdir(Path)
Datasets_features = {}
for Filename in glob('*.csv'):
    Datasets_features[Filename[:-4].title()] = pd.read_csv(Filename, sep = ',')
    
#Defining variables.
Train_ftr = Datasets_features['Train']
Validation_ftr = Datasets_features['Validation']

#### Define general functions

In [28]:

#Linear bidding strategy.
def linear_bidding(base_bid, pCTR, Init_CTR):
    return (base_bid*pCTR/Init_CTR)

def nlogn_bidding(base_bid, pCTR, Init_CTR):
    return (base_bid*(pCTR/Init_CTR)*np.log(pCTR/Init_CTR))

def exp_bidding(base_bid, pCTR, Init_CTR):
    return (base_bid*np.e**(pCTR/Init_CTR))


#Winning criterion #1.
def win_auction_1(bid, payprice):
    return bid >= payprice
    
#Winning criterion #2.
def win_auction_2(bid, payprice, Others):
    return ((bid >= payprice) and (bid >= max(Others)))
    
#Bidding strategy parameter eveluation.
def Bidding_strategy_prm(Budget, Init_CTR, Init_eCPC, Clicks_Prices, 
                         Pred_CTRs, prm, Dict_alg, key, curr_strat_vals,alg):
    
    empCPM = []
    Bids, Cost, Imps, Clks = 0, 0, 0, 0
    
    #Looping on bid requests.
    for item in range(0, len(Clicks_Prices)):
        bid = 0
        Bids += 1
        pCTR = Pred_CTRs[item]           
            
        #Linear strategy.
        if key == 'Linear':
            Criterion = 1
            bid = linear_bidding(prm, pCTR, Init_CTR)
            Win = win_auction_1(bid, Clicks_Prices[item][1])
        # NLogN strategy 
        elif key == 'NLogN':
            Criterion = 1
            bid = nlogn_bidding(prm, pCTR, Init_CTR)
            Win = win_auction_1(bid, Clicks_Prices[item][1])
        elif key == 'Exp':
            Criterion = 1
            bid = exp_bidding(prm, pCTR, Init_CTR)
            Win = win_auction_1(bid, Clicks_Prices[item][1])
            
        #Exception: error.
        else:
            print('[Strategy error]: Wrong bidding strategy name!')
            break
            
        #Computing bid outcome.
        if Win:
            Imps += 1
            Clks += Clicks_Prices[item][0]
            
            if Criterion == 1:
                Cost += Clicks_Prices[item][1]/1000
                empCPM.append(Clicks_Prices[item][1])
            else:
                Cost += max(Clicks_Prices[item][1],max(others))/1000
                empCPM.append(max(Clicks_Prices[item][1],max(others)))
                
        #Stopping condition.
        if Cost > Budget:
            break
            
    #Reporting results.
    if Imps != 0: empCTR = Clks/Imps
    else: empCTR = 0
    
    avgCPM = np.mean(empCPM)
    if Clks != 0: avgCPC = np.sum(empCPM)/(Clks*1000)
    else: avgCPC = 0
    
    Output =  key + ',' + alg + ',' + str(prm) + ',' + str(Budget) + ',' + str(Cost) + ','\
              + str(Bids) + ',' + str(Imps) + ',' + str(Clks) + ','\
              + str(empCTR) + ',' + str(avgCPM) + ',' + str(avgCPC)
    #temp_df = pd.DataFrame(,columns=['Algorithm', 'Parameter', 'Budget','Spent','Imps', 'Click','eCTR','avgCPM','avgCPC'])
    curr_strat_vals.loc[len(curr_strat_vals)]=[key,alg,str(prm),Budget,Cost,Bids,Imps,Clks,empCTR,avgCPM,avgCPC]
    return Output
    
#Bidding strategy simulation.
def Bidding_strategy(Budget, Init_CTR, Init_eCPC, Clicks_Prices, Pred_CTRs,
                     file, Dict_alg, key, curr_strat_vals, alg):
    parameters = Dict_alg[key]
    for prm in parameters:
        Output = Bidding_strategy_prm(Budget, Init_CTR, Init_eCPC, Clicks_Prices,
                                      Pred_CTRs, prm, Dict_alg, key, curr_strat_vals, alg)
        file.write(Output + '\n')

#Actual bid creation and writing into file, based on baseB
def createBids(bidIds,baseBid,Pred_CTRs,Init_CTR,alg_name ):
    Filename = 'Team_07_'+ alg_name + '_bids' +'.csv'
    print('Using best param...' + '{:.3f}'.format(baseBid))
    with open(Filename, 'w') as file:
        file.write('bidid,bidprice' + '\n')
        for idx in range(len(bidIds)):
            bidId = bidIds[idx]
            pCTR  = Pred_CTRs[idx]
            bid   = linear_bidding(baseBid,pCTR, Init_CTR)
            file.write(bidId + ',' + '{:.1f}'.format(bid) +'\n')
    print('• [File %s]: Process completed!' %Filename)

#### Read CTR information, base bid optimisation and bid creation
For CTR, two files should be in the folder, with the next format: '<Algorithm\>\_test\_pCTR.csv' and '<Algorithm\>\_validation\_pCTR.csv' (Note: Currently case sensitive). Then, the algorithm name shall be included in the _algs_ array.

In [46]:
#Defining CTR directory.
Path = 'C://Datasets//Multiagent//calc'

#Importing data.
os.chdir(Path)
Datasets_CTR = {}
overall_alg_vals = pd.DataFrame(columns=['BidStrat','Algorithm', 'Parameter', 'Budget','Spent','Bids','Imps', 'Click','eCTR','avgCPM','avgCPC'])

for Filename in glob('*_pCTR.csv'):
    Datasets_CTR[Filename[:-4]] = pd.read_csv(Filename, sep = ',')

#Define algorithms to use
algs = ['Catboost','LGBM','LR']



#Initialising values.
Init_CTR = sum(Train_ftr['click'])/len(Train_ftr)
Init_eCPC = sum(Train_ftr['payprice'])/(sum(Train_ftr['click'])*1000)
print("avgCTR: ",Init_CTR)
    
for alg in algs:
    #Defining variables.
    Test_CTR = Datasets_CTR[alg+'_test_pCTR']
    Validation_CTR = Datasets_CTR[alg+'_validation_pCTR']
    #Training bidding strategies.
    Budget = 6250
    random.seed(10)

    #Path = 'C://Users//mau_c//OneDrive//Documentos//UCL//MultiAgent//Untitled Folder'

    #Defining general variables.
    Pred_CTRs = []
    Clicks_Prices = []

    #Initialising parameters.
    Prm_lin = np.concatenate((np.arange(2, 20, 2),np.arange(20, 100, 5),
                              np.arange(100, 400, 10), np.arange(400, 800, 50)),
                              axis=0)
    Prm_log = np.logspace(2.8,4,num=20)

    Dict_alg = {'Linear': Prm_lin,  'Exp': Prm_lin}#'NLogN': Prm_log,
    
    Pred_CTRs = list(Validation_CTR['pCTR'].values)
    Pred_CTRs_Test = list(Test_CTR['pCTR'].values)
    Clicks_Prices = list(zip(Validation_ftr['click'], Validation_ftr['payprice']))

    #Training and saving results.
    os.chdir(Path)

    Header = 'bidStrat,algorithm,parameter,budget,spend,bids,imps,click,empCTR,avgCPM,avgCPC'

    for key in Dict_alg.keys():
        print('Processing: ', alg,'-',key)
        Filename = key + '_BB_'+ alg+'.csv'
        curr_alg_vals = pd.DataFrame(columns=['BidStrat','Algorithm', 'Parameter', 'Budget','Spent','Bids', 'Imps','Click','eCTR','avgCPM','avgCPC'])
        with open(Filename, 'w') as file:
            file.write(Header + '\n')
            Bidding_strategy(Budget, Init_CTR, Init_eCPC, Clicks_Prices,
                             Pred_CTRs, file, Dict_alg, key, curr_alg_vals, alg)
        print('• [File %s]: Process completed!' %Filename)
        overall_alg_vals = overall_alg_vals.append(curr_alg_vals.sort_values('Click', ascending=False).head(2), sort=False)
        bestParam = np.single(curr_alg_vals.sort_values('Click', ascending=False).head(1).iloc[0,2])
        createBids(Test_CTR['bidid'],bestParam,Pred_CTRs_Test,Init_CTR,alg)

print('• Full process completed!')
print(overall_alg_vals.sort_values('Click', ascending=False))

avgCTR:  0.0007375623256619447
Processing:  Catboost - Linear
• [File Linear_BB_Catboost.csv]: Process completed!
Using best param...90.000
• [File Team_07_Catboost_bids.csv]: Process completed!
Processing:  Catboost - Exp
• [File Exp_BB_Catboost.csv]: Process completed!
Using best param...25.000
• [File Team_07_Catboost_bids.csv]: Process completed!
Processing:  LGBM - Linear
• [File Linear_BB_LGBM.csv]: Process completed!
Using best param...120.000
• [File Team_07_LGBM_bids.csv]: Process completed!
Processing:  LGBM - Exp
• [File Exp_BB_LGBM.csv]: Process completed!
Using best param...35.000
• [File Team_07_LGBM_bids.csv]: Process completed!
Processing:  LR - Linear
• [File Linear_BB_LR.csv]: Process completed!
Using best param...95.000
• [File Team_07_LR_bids.csv]: Process completed!
Processing:  LR - Exp


C:\Users\mau_c\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: RuntimeWarning: overflow encountered in double_scalars
  # Remove the CWD from sys.path while we load stuff.


• [File Exp_BB_LR.csv]: Process completed!
Using best param...30.000
• [File Team_07_LR_bids.csv]: Process completed!
• Full process completed!
   BidStrat Algorithm Parameter Budget     Spent    Bids    Imps Click  \
27   Linear      LGBM       120   6250  6204.623  303925  140033   161   
26   Linear      LGBM       110   6250  5628.472  303925  131711   158   
12      Exp      LGBM        35   6250  5930.920  303925  145755   156   
24   Linear        LR        95   6250  5974.248  303925  118504   155   
25   Linear        LR       100   6250  6250.039  301401  122309   155   
11      Exp      LGBM        30   6250  4876.103  303925  128945   152   
11      Exp        LR        30   6250  6229.484  303925  138977   152   
10      Exp        LR        25   6250  5116.817  303925  120446   149   
23   Linear  Catboost        90   6250  6250.051  301477  124505   145   
22   Linear  Catboost        85   6250  5900.820  303925  119605   144   
10      Exp  Catboost        25   6250  57

In [47]:
#np.concatenate((np.arange(2, 20, 2),np.arange(20, 100, 5),
                        #      np.arange(100, 400, 10), np.arange(400, 800, 50)),
                         #     axis=0)
overall_alg_vals.sort_values('Click', ascending=False)

,BidStrat,Algorithm,Parameter,Budget,Spent,Bids,Imps,Click,eCTR,avgCPM,avgCPC
27,Linear,LGBM,120,6250,6204.623,303925,140033,161,0.001150,44.308292,38.538031
26,Linear,LGBM,110,6250,5628.472,303925,131711,158,0.001200,42.733500,35.623241
12,Exp,LGBM,35,6250,5930.920,303925,145755,156,0.001070,40.691023,38.018718
24,Linear,LR,95,6250,5974.248,303925,118504,155,0.001308,50.413893,38.543535
25,Linear,LR,100,6250,6250.039,301401,122309,155,0.001267,51.100401,40.322832
11,Exp,LGBM,30,6250,4876.103,303925,128945,152,0.001179,37.815371,32.079625
11,Exp,LR,30,6250,6229.484,303925,138977,152,0.001094,44.823849,40.983447
10,Exp,LR,25,6250,5116.817,303925,120446,149,0.001237,42.482249,34.341054
23,Linear,Catboost,90,6250,6250.051,301477,124505,145,0.001165,50.199197,43.103800
22,Linear,Catboost,85,6250,5900.820,303925,119605,144,0.001204,49.335897,40.977917


In [48]:
import winsound
duration = 1000  # milliseconds
freq = 440  # Hz
winsound.Beep(freq, duration)